In [3]:
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import os

import torch
from torch.utils.data import Dataset
import torch.optim as optim
import torch.nn as nn
from torchvision import transforms

from sklearn.metrics import classification_report, fbeta_score, matthews_corrcoef
from tqdm import tqdm
import torch.nn.functional as F

In [4]:
# Define hyperparameters
BATCH_SIZE = 86
LEARNING_RATE = 0.0001

In [5]:
# Set the device
device = "cpu"
if torch.cuda.is_available():
    device =  "cuda"
elif torch.backends.mps.is_available():
    device = "mps" # Use M1 Mac GPU
    device="cpu"
print(device)

cpu


In [6]:
class ExtractorClassifierDataset(Dataset):
    def __init__(self, data, resize_shape=(256, 256)):
        self.data = data
        self.to_tensor = transforms.ToTensor()
        self.resize_image = transforms.Resize(resize_shape, antialias=True)
        self.image_tensors = []
        self.label_tensors = []

        for image, label in tqdm(self.data):
            # Apply transformations to the image and convert to tensor
            image_tensor = self.to_tensor(image)
            image_tensor = self.resize_image(image_tensor)
            image_tensor = image_tensor/255
            self.image_tensors.append(image_tensor.to(device))

            # Create one-hot encoded tensor for the label
            label_tensor = torch.zeros(2)
            label_tensor[label] = 1
            self.label_tensors.append(label_tensor.to(device))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.image_tensors[idx], self.label_tensors[idx]

data_dir = "../../../datasets/extractor_classifier"

train_list = []
validation_list = []
test_list = []

for class_name in ("relevant", "not_relevant"):
    if class_name == "relevant":
        label = 0
    else:
        label = 1

    for image in os.listdir(f"{data_dir}/train/{class_name}"):
        if image != ".ipynb_checkpoints":
            src_path = f"{data_dir}/train/{class_name}/{image}"
            train_list.append((Image.open(src_path), label))

# Uncomment this part to add augmented data
    for image in os.listdir(f"{data_dir}/train_data_augmentation/{class_name}"):
        if image != ".ipynb_checkpoints":
            src_path = f"{data_dir}/train_data_augmentation/{class_name}/{image}"
            train_list.append((Image.open(src_path), label))

    for image in os.listdir(f"{data_dir}/validation/{class_name}"):
        if image != ".ipynb_checkpoints":
            src_path = f"{data_dir}/validation/{class_name}/{image}"
            validation_list.append((Image.open(src_path), label))

    for image in os.listdir(f"{data_dir}/test/{class_name}"):
        if image != ".ipynb_checkpoints":
            src_path = f"{data_dir}/test/{class_name}/{image}"
            test_list.append((Image.open(src_path), label))

# Define the train/validation/test datasets
train_data = ExtractorClassifierDataset(train_list)
validation_data = ExtractorClassifierDataset(validation_list)
test_data = ExtractorClassifierDataset(test_list)

100%|██████████| 698/698 [00:04<00:00, 146.72it/s]


In [7]:
import gc
del train_list
del validation_list
del test_list
gc.collect()

433

In [7]:
# Define the dataloaders for each dataset
train_loader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, pin_memory=False )
validation_loader = torch.utils.data.DataLoader(validation_data, batch_size=BATCH_SIZE, shuffle=True, pin_memory=False)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True, pin_memory=False)

In [7]:
from torchvision.models import resnet50

class Resnet50Model(nn.Module):
    def __init__(self, pretrained=False):
        super(Resnet50Model, self).__init__()
        self.resnet_model = resnet50(pretrained=pretrained)
        num_features = self.resnet_model.fc.in_features
        self.resnet_model.fc = nn.Linear(num_features, 2)
        
    def forward(self, x):
        x = self.resnet_model(x)
        return x

In [9]:
from torchvision.models import convnext_base

class ConvNeXtBaseModel(nn.Module):
    def __init__(self, pretrained=False):
        super(ConvNeXtBaseModel, self).__init__()
        self.convnext_model = convnext_base(pretrained=pretrained)
        num_features = self.convnext_model.classifier[2].in_features
        new_classifier_layer = torch.nn.Linear(num_features, 2)
        self.convnext_model.classifier[2] = new_classifier_layer
        
    def forward(self, x):
        x = self.convnext_model(x)
        return x

In [8]:
from torchvision.models import efficientnet_v2_m

class EfficientNetModel(nn.Module):
    def __init__(self, pretrained=False):
        super(EfficientNetModel, self).__init__()
        self.efficientnet_model = efficientnet_v2_m(pretrained=pretrained)
        num_features = self.efficientnet_model.classifier[1].in_features
        new_classifier_layer = torch.nn.Linear(num_features, 2)
        self.efficientnet_model.classifier[1] = new_classifier_layer
        
    def forward(self, x):
        x = self.efficientnet_model(x)
        return x

In [9]:
def train(model, model_path, train_loader, validation_loader, test_loader, criterion, optimizer, epochs):
    # Move the model to the device
    model.to(device)

    # Define variables to track the best validation accuracy and the corresponding model state
    best_valid_f_beta = 0.0
    train_loss_values = []
    validation_loss_values = []
    train_f_beta_values = []
    validation_f_beta_values = []

    train_true_labels = []
    train_predicted_labels = []

    for epoch in range(epochs):
        # Train the model for one epoch
        train_loss = 0.0
        model.train()
        for i, (images, labels) in enumerate(tqdm(train_loader)):
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            train_loss += loss.item() * images.size(0)
            loss.backward()

            #if (i + 1) % 2 == 0 or (i + 1) == len(train_loader):
            optimizer.step()
            optimizer.zero_grad()

            outputs = F.softmax(outputs, dim=1)
            _, predicted = torch.max(outputs.data, 1)
            _, labels = torch.max(labels.data, 1)

            train_predicted_labels.extend(predicted.detach().cpu())
            train_true_labels.extend(labels.detach().cpu())

        train_loss = train_loss / len(train_loader.dataset)
        train_loss_values.append(train_loss)

        train_f_beta = fbeta_score(train_true_labels, train_predicted_labels, average="binary", beta=0.5)
        train_f_beta_values.append(train_f_beta)

        print(f"Epoch {epoch+1}/{epochs} - "
              f"Train Loss: {train_loss:.4f} - Train Fbeta: {train_f_beta:.4f}")

        # Evaluate the model on the validation set
        validation_loss, validation_f_beta = validate_model(model, validation_loader)
        validation_loss_values.append(validation_loss)
        validation_f_beta_values.append(validation_f_beta)

        # Save the model state if the current validation accuracy is better than the previous best
        if validation_f_beta > best_valid_f_beta:
            best_valid_f_beta = validation_f_beta
            print(f"Updated best model: epoch {epoch+1}")
            torch.save(model.state_dict(), model_path)

    # Load the best model state and evaluate on the test set
    model.load_state_dict(torch.load(model_path, map_location=torch.device(device)))
    print("Test model:")
    test_loss, test_f_beta = validate_model(model, test_loader)
    print(f"Test Loss: {test_loss:.4f} - Test Fbeta: {test_f_beta:.4f}")

    return model, train_loss_values, train_f_beta_values, validation_loss_values, validation_f_beta_values

def validate_model(model, data_loader):
    model.to(device)
    model.eval()

    valid_loss = 0.0
    valid_true_labels = []
    valid_predicted_labels = []

    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)

            loss = criterion(outputs, labels)
            valid_loss += loss.item() * images.size(0)

            outputs = F.softmax(outputs, dim=1)
            _, predicted = torch.max(outputs.data, 1)
            _, labels = torch.max(labels.data, 1)

            valid_predicted_labels.extend(predicted.detach().cpu())
            valid_true_labels.extend(labels.detach().cpu())

    f_beta_score = fbeta_score(valid_true_labels, valid_predicted_labels, average="binary", beta=0.5)
    mcc_score = matthews_corrcoef(valid_true_labels, valid_predicted_labels)
    valid_loss = valid_loss / len(data_loader.dataset)

    print(f"Valid Loss: {valid_loss:.4f} - Valid Fbeta: {f_beta_score:.4f} - Valid MCC: {mcc_score}")
    print(classification_report(valid_true_labels, valid_predicted_labels))
    return valid_loss, f_beta_score

In [25]:
import gc
del train_loader
del validation_loader
del test_loader
gc.collect()

25067

In [10]:
efficient_net_model_augmented_data = EfficientNetModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(efficient_net_model_augmented_data.parameters(), lr=LEARNING_RATE)
model_path = "model_results/efficient_net_augmented_model_50.pt"

if os.path.isfile(model_path):
    print("Evaluate model")
    efficient_net_model_augmented_data.load_state_dict(torch.load(model_path, map_location=torch.device(device)))
    print("Validation data:")
    validate_model(efficient_net_model_augmented_data, validation_loader)
    print("Test data:")
    validate_model(efficient_net_model_augmented_data, test_loader)
else:
    print("Train model")
    model, train_loss_values, train_f_beta_values, validation_loss_values, validation_f_beta_values = train(efficient_net_model_augmented_data, model_path, train_loader, validation_loader, test_loader, criterion, optimizer, 50)
    np.save("model_results/efficient_net_augmented_model_50_train_loss", train_loss_values)
    np.save("model_results/efficient_net_augmented_model_50_train_f_beta", train_f_beta_values)
    np.save("model_results/efficient_net_augmented_model_50_valid_loss", validation_loss_values)
    np.save("model_results/efficient_net_augmented_model_50_valid_f_beta", validation_f_beta_values)

/Users/I518152/opt/miniconda3/envs/ankinator/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/I518152/opt/miniconda3/envs/ankinator/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Train model


  0%|          | 0/85 [03:45<?, ?it/s]


KeyboardInterrupt: 

In [11]:
resnet_model = Resnet50Model()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet_model.parameters(), lr=LEARNING_RATE)
model_path = "model_results/resnet50_model_50.pt"

if os.path.isfile(model_path):
    print("Evaluate model")
    resnet_model.load_state_dict(torch.load(model_path, map_location=torch.device(device)))
    print("Validation data:")
    validate_model(resnet_model, validation_loader)
    print("Test data:")
    validate_model(resnet_model, test_loader)
else:
    print("Train model")
    model, train_loss_values, train_f_beta_values, validation_loss_values, validation_f_beta_values = train(resnet_model, model_path, train_loader, validation_loader, test_loader, criterion, optimizer, 50)
    np.save("model_results/resnet50_model_50_train_loss", train_loss_values)
    np.save("model_results/resnet50_model_50_train_f_beta", train_f_beta_values)
    np.save("model_results/resnet50_model_50_valid_loss", validation_loss_values)
    np.save("model_results/resnet50_model_50_valid_f_beta", validation_f_beta_values)

/home/ma/ma_ma/ma_nikemper/.local/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/ma/ma_ma/ma_nikemper/.local/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Train model


100%|██████████| 29/29 [00:13<00:00,  2.15it/s]


Epoch 1/50 - Train Loss: 0.5801 - Train Fbeta: 0.5963


/opt/bwhpc/common/jupyter/tensorflow/2023-03-24/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/bwhpc/common/jupyter/tensorflow/2023-03-24/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/bwhpc/common/jupyter/tensorflow/2023-03-24/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf

...

Epoch 48/50 - Train Loss: 0.0168 - Train Fbeta: 0.9518
Valid Loss: 1.4202 - Valid Fbeta: 0.8039 - Valid MCC: 0.6201797046499385
              precision    recall  f1-score   support

           0       0.81      0.94      0.87       218
           1       0.86      0.63      0.73       130

    accuracy                           0.82       348
   macro avg       0.84      0.79      0.80       348
weighted avg       0.83      0.82      0.82       348



100%|██████████| 29/29 [00:10<00:00,  2.88it/s]


Epoch 49/50 - Train Loss: 0.0132 - Train Fbeta: 0.9528
Valid Loss: 1.1211 - Valid Fbeta: 0.7235 - Valid MCC: 0.5473919635868973
              precision    recall  f1-score   support

           0       0.82      0.85      0.84       218
           1       0.73      0.69      0.71       130

    accuracy                           0.79       348
   macro avg       0.78      0.77      0.77       348
weighted avg       0.79      0.79      0.79       348



100%|██████████| 29/29 [00:10<00:00,  2.89it/s]


Epoch 50/50 - Train Loss: 0.0333 - Train Fbeta: 0.9535
Valid Loss: 1.9468 - Valid Fbeta: 0.5648 - Valid MCC: 0.39637158366839337
              precision    recall  f1-score   support

           0       0.88      0.51      0.65       218
           1       0.52      0.88      0.65       130

    accuracy                           0.65       348
   macro avg       0.70      0.70      0.65       348
weighted avg       0.75      0.65      0.65       348

Test model:
Valid Loss: 1.5313 - Valid Fbeta: 0.8054 - Valid MCC: 0.6043441825539875
              precision    recall  f1-score   support

           0       0.79      0.96      0.87       436
           1       0.90      0.57      0.70       261

    accuracy                           0.81       697
   macro avg       0.84      0.77      0.78       697
weighted avg       0.83      0.81      0.80       697

Test Loss: 1.5313 - Test Fbeta: 0.8054


In [15]:
efficient_net_model = EfficientNetModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(efficient_net_model.parameters(), lr=LEARNING_RATE)
model_path = "model_results/efficient_net50_model_50.pt"

if os.path.isfile(model_path):
    print("Evaluate model")
    efficient_net_model.load_state_dict(torch.load(model_path, map_location=torch.device(device)))
    print("Validation data:")
    validate_model(efficient_net_model, validation_loader)
    print("Test data:")
    validate_model(efficient_net_model, test_loader)
else:
    print("Train model")
    model, train_loss_values, train_f_beta_values, validation_loss_values, validation_f_beta_values = train(efficient_net_model, model_path, train_loader, validation_loader, test_loader, criterion, optimizer, 50)
    np.save("model_results/efficient_net50_model_50_train_loss", train_loss_values)
    np.save("model_results/efficient_net50_model_50_train_f_beta", train_f_beta_values)
    np.save("model_results/efficient_net50_model_50_valid_loss", validation_loss_values)
    np.save("model_results/efficient_net50_model_50_valid_f_beta", validation_f_beta_values)

Train model


100%|██████████| 29/29 [00:38<00:00,  1.32s/it]


Epoch 1/50 - Train Loss: 0.6851 - Train Fbeta: 0.3632


/opt/bwhpc/common/jupyter/tensorflow/2023-03-24/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/bwhpc/common/jupyter/tensorflow/2023-03-24/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/bwhpc/common/jupyter/tensorflow/2023-03-24/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf

Valid Loss: 0.6808 - Valid Fbeta: 0.0000 - Valid MCC: 0.0
              precision    recall  f1-score   support

           0       0.63      1.00      0.77       218
           1       0.00      0.00      0.00       130

    accuracy                           0.63       348
   macro avg       0.31      0.50      0.39       348
weighted avg       0.39      0.63      0.48       348



...

Epoch 48/50 - Train Loss: 0.0354 - Train Fbeta: 0.9119
Valid Loss: 0.7657 - Valid Fbeta: 0.7778 - Valid MCC: 0.6352258593817761
              precision    recall  f1-score   support

           0       0.86      0.88      0.87       218
           1       0.78      0.75      0.77       130

    accuracy                           0.83       348
   macro avg       0.82      0.81      0.82       348
weighted avg       0.83      0.83      0.83       348



100%|██████████| 29/29 [00:18<00:00,  1.56it/s]


Epoch 49/50 - Train Loss: 0.0168 - Train Fbeta: 0.9137
Valid Loss: 0.9510 - Valid Fbeta: 0.7673 - Valid MCC: 0.6079128800539787
              precision    recall  f1-score   support

           0       0.84      0.88      0.86       218
           1       0.78      0.72      0.75       130

    accuracy                           0.82       348
   macro avg       0.81      0.80      0.80       348
weighted avg       0.82      0.82      0.82       348



100%|██████████| 29/29 [00:18<00:00,  1.56it/s]


Epoch 50/50 - Train Loss: 0.0232 - Train Fbeta: 0.9154
Valid Loss: 0.7513 - Valid Fbeta: 0.7980 - Valid MCC: 0.6585642862164827
              precision    recall  f1-score   support

           0       0.86      0.89      0.88       218
           1       0.81      0.75      0.78       130

    accuracy                           0.84       348
   macro avg       0.83      0.82      0.83       348
weighted avg       0.84      0.84      0.84       348

Test model:
Valid Loss: 0.7006 - Valid Fbeta: 0.8163 - Valid MCC: 0.6376924519454447
              precision    recall  f1-score   support

           0       0.82      0.94      0.88       436
           1       0.88      0.64      0.74       261

    accuracy                           0.83       697
   macro avg       0.85      0.79      0.81       697
weighted avg       0.84      0.83      0.83       697

Test Loss: 0.7006 - Test Fbeta: 0.8163


In [8]:
convnext_model = ConvNeXtBaseModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(convnext_model.parameters(), lr=LEARNING_RATE)
model_path = "model_results/convnext_model_50.pt"

if os.path.isfile(model_path):
    print("Evaluate model")
    convnext_model.load_state_dict(torch.load(model_path, map_location=torch.device(device)))
    print("Validation data:")
    validate_model(convnext_model, validation_loader)
    print("Test data:")
    validate_model(convnext_model, test_loader)
else:
    print("Train model")
    model, train_loss_values, train_f_beta_values, validation_loss_values, validation_f_beta_values = train(convnext_model, model_path, train_loader, validation_loader, test_loader, criterion, optimizer, 50)
    np.save("model_results/convnext_model_50_train_loss", train_loss_values)
    np.save("model_results/convnext_model_50_train_f_beta", train_f_beta_values)
    np.save("model_results/convnext_model_50_valid_loss", validation_loss_values)
    np.save("model_results/convnext_model_50_valid_f_beta", validation_f_beta_values)

/home/ma/ma_ma/ma_nikemper/.local/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/ma/ma_ma/ma_nikemper/.local/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Train model


100%|██████████| 29/29 [00:56<00:00,  1.94s/it]


Epoch 1/50 - Train Loss: 0.8413 - Train Fbeta: 0.3498
Valid Loss: 0.6880 - Valid Fbeta: 0.5025 - Valid MCC: 0.22555228909385755
              precision    recall  f1-score   support

           0       0.68      0.88      0.77       218
           1       0.60      0.31      0.41       130

    accuracy                           0.66       348
   macro avg       0.64      0.59      0.59       348
weighted avg       0.65      0.66      0.63       348

Updated best model: epoch 1


100%|██████████| 29/29 [00:54<00:00,  1.89s/it]


...

Epoch 48/50 - Train Loss: 0.1125 - Train Fbeta: 0.7615
Valid Loss: 0.7604 - Valid Fbeta: 0.7632 - Valid MCC: 0.586520650811533
              precision    recall  f1-score   support

           0       0.82      0.89      0.86       218
           1       0.79      0.67      0.72       130

    accuracy                           0.81       348
   macro avg       0.81      0.78      0.79       348
weighted avg       0.81      0.81      0.81       348



100%|██████████| 29/29 [00:55<00:00,  1.90s/it]


Epoch 49/50 - Train Loss: 0.1112 - Train Fbeta: 0.7664
Valid Loss: 0.7442 - Valid Fbeta: 0.7840 - Valid MCC: 0.6185774219369887
              precision    recall  f1-score   support

           0       0.83      0.90      0.87       218
           1       0.81      0.69      0.75       130

    accuracy                           0.82       348
   macro avg       0.82      0.80      0.81       348
weighted avg       0.82      0.82      0.82       348



100%|██████████| 29/29 [00:55<00:00,  1.90s/it]


Epoch 50/50 - Train Loss: 0.0870 - Train Fbeta: 0.7715
Valid Loss: 0.8488 - Valid Fbeta: 0.6972 - Valid MCC: 0.5401922268039268
              precision    recall  f1-score   support

           0       0.85      0.79      0.82       218
           1       0.68      0.76      0.72       130

    accuracy                           0.78       348
   macro avg       0.77      0.78      0.77       348
weighted avg       0.79      0.78      0.78       348

Test model:
Valid Loss: 0.5655 - Valid Fbeta: 0.7274 - Valid MCC: 0.5333354092181162
              precision    recall  f1-score   support

           0       0.80      0.88      0.84       436
           1       0.75      0.64      0.69       261

    accuracy                           0.79       697
   macro avg       0.78      0.76      0.76       697
weighted avg       0.78      0.79      0.78       697

Test Loss: 0.5655 - Test Fbeta: 0.7274
